In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

import utils
import plot_utils

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


# Read data

In [2]:
## Skeletal data

data_file = './data/paleo_meso_ceno_data_paleogeography_epoch.csv'
data = pd.read_csv(data_file)
data = data.sort_values(by=['mean_ma'], ascending=False)

time_file = './data/timeScale.xlsx'
time = pd.read_excel(time_file,sheet_name='Sheet1')
time_period = utils.get_timesubset(time,3,541)
width_period = [time_period['max_ma'][i] - time_period['min_ma'][i] for i in list(time_period.index)]

time_epoch = utils.get_timesubset(time,4,541)
width_epoch = [time_epoch['max_ma'][i] - time_epoch['min_ma'][i] for i in list(time_epoch.index)]

In [3]:
#Animal biomass
animals = {'brachiopoda':['brachiopods'],
 'molluska':['gastropods','bivalves','molluska','cephalopods'],
 'echinodermata':['echinoids', 'crinoids','echinoderm'],
 'porifera':['sponges', 'sponges_silica','sponges_archaeocyath', 'sponges_calc'],
 'cnidaria':['sclerec_corals','rugose_corals', 'corals'],
 'arthropoda':['trilobite', 'ostracods', 'barnacles'],
          'bryozoa':['bryozoans']}


time_scale = 'epoch'

unique_time = np.unique(data[time_scale])
abundance_mean = []
abundance_time = []
abundance_animal = []
for t in unique_time:
    temp = data[data[time_scale] == t].copy()
    for a in animals.keys():
        total_abundance = temp[animals[a]].sum(axis=1).sum()
        total_samples = temp['numberOfSamples'].sum()
        abundance_mean.append(total_abundance/total_samples)
        abundance_animal.append(a)
        abundance_time.append(t)
        
df = pd.DataFrame({'animal':abundance_animal,'time':abundance_time,'abundance':abundance_mean})

df = utils.get_mean_time(time_epoch,df,'time')
unique_time = df.time.unique()

abundance_percent = []
for t in unique_time:
    temp = df[df.time == t]
    for _,dat in temp.iterrows():
        abundance_percent.append(100*dat.abundance/temp.abundance.sum())

df['abundance_percent'] = abundance_percent


In [4]:
time_scale = 'epoch'

data_animal = utils.get_stats(data,'total_animals_new',time_scale)
data_protist = utils.get_stats(data,'total_protists_new',time_scale)
data_algamicro = utils.get_stats(data,'total_alga_new',time_scale)

data_animal = data_animal.sort_values(by=['time'], ascending=True)
data_protist = data_protist.sort_values(by=['time'], ascending=True)
data_algamicro = data_algamicro.sort_values(by=['time'], ascending=True)

data_animal['skeletal_grain'] = 'animal'
data_protist['skeletal_grain'] = 'protist'
data_algamicro['skeletal_grain'] = 'algamicro'

data_skeletalgrain = pd.concat([data_animal,data_protist, data_algamicro])
data_skeletalgrain = utils.get_mean_time(time_epoch,data_skeletalgrain,time_scale)
unique_time_data = data_skeletalgrain[time_scale].unique()

data_abundance_percent = []
for t in unique_time_data:
    temp = data_skeletalgrain[data_skeletalgrain[time_scale] == t]
    for _,dat in temp.iterrows():
        data_abundance_percent.append(100*dat['mean']/temp['mean'].sum())

data_skeletalgrain['abundance_percent'] = data_abundance_percent

# Occurrence all litho

In [5]:
animal_occurrence, algae_occurrence, protist_occurrence = utils.load_occurrence_data()

In [6]:
skeletal_occurrence, animal_relative_occurrence_phylum = utils.get_skeletal_animal_phyllum_occurrence(animal_occurrence, algae_occurrence, protist_occurrence,time_epoch)

In [7]:
fig = make_subplots(rows = 4,cols = 1,shared_xaxes = True,vertical_spacing = 0.05,
                    subplot_titles=['<b>Skeletal biomass', '<b>Skeletal occurrence',
                                   '<b>Animal phylum biomass', '<b>Animal phylum occurrence',]
                   )

colors = px.colors.qualitative.Safe


fig.add_trace(go.Bar(x=data_skeletalgrain[data_skeletalgrain.skeletal_grain == 'animal'][time_scale],
                     y=data_skeletalgrain[data_skeletalgrain.skeletal_grain == 'animal']['abundance_percent'],
                     marker_color=colors[0],legendgroup='2',legendgrouptitle_text='Skeletal grains',
                     name='Animal'),row=1,col=1)

fig.add_trace(go.Bar(x=data_skeletalgrain[data_skeletalgrain.skeletal_grain == 'protist'][time_scale],
                     y=data_skeletalgrain[data_skeletalgrain.skeletal_grain == 'protist']['abundance_percent'],
                     marker_color=colors[1],legendgroup='2',
                     name='Protist'),row=1,col=1)

fig.add_trace(go.Bar(x=data_skeletalgrain[data_skeletalgrain.skeletal_grain == 'algamicro'][time_scale],
                     y=data_skeletalgrain[data_skeletalgrain.skeletal_grain == 'algamicro']['abundance_percent'],
                     marker_color=colors[2],legendgroup='2',
                     name='Algae'),row=1,col=1)


fig.add_trace(go.Bar(x=skeletal_occurrence['time'],
                     y=skeletal_occurrence['animal'],
                     marker_color=colors[0],
                     showlegend=False,),row=2,col=1)

fig.add_trace(go.Bar(x=skeletal_occurrence['time'],
                     y=skeletal_occurrence['protist'],
                     showlegend=False,
                     marker_color=colors[1],),row=2,col=1)

fig.add_trace(go.Bar(x=skeletal_occurrence['time'],
                     y=skeletal_occurrence['algae'],
                     showlegend=False,
                     marker_color=colors[2],),row=2,col=1)


colors = px.colors.sequential.Turbo_r

fig.add_trace(go.Bar(x=df[df.animal == 'echinodermata']['time'],
                     y=df[df.animal == 'echinodermata']['abundance_percent'],marker_color=colors[2],
                    legendgroup='1',legendgrouptitle_text='Animal phyla',
                     name='Echinodermata'),row=3,col=1)

fig.add_trace(go.Bar(x=df[df.animal == 'bryozoa']['time'],
                     y=df[df.animal == 'bryozoa']['abundance_percent'],marker_color=colors[4],
                     legendgroup='1',
                     name='Bryozoa'),row=3,col=1)

fig.add_trace(go.Bar(x=df[df.animal == 'brachiopoda']['time'],
                     y=df[df.animal == 'brachiopoda']['abundance_percent'],marker_color=colors[6],
                     legendgroup='1',
                     name='Brachiopoda'),row=3,col=1)

fig.add_trace(go.Bar(x=df[df.animal == 'molluska']['time'],
                     y=df[df.animal == 'molluska']['abundance_percent'],marker_color='#065535',
                     legendgroup='1',
                    name='Mollusca'),row=3,col=1)

fig.add_trace(go.Bar(x=df[df.animal == 'porifera']['time'],
                     y=df[df.animal == 'porifera']['abundance_percent'],marker_color=colors[10],
                     legendgroup='1',
                     name='Porifera'),row=3,col=1)

fig.add_trace(go.Bar(x=df[df.animal == 'cnidaria']['time'],
                     y=df[df.animal == 'cnidaria']['abundance_percent'],marker_color=colors[12],
                     legendgroup='1',
                     name='Cnidaria'),row=3,col=1)

fig.add_trace(go.Bar(x=df[df.animal == 'arthropoda']['time'],
                     y=df[df.animal == 'arthropoda']['abundance_percent'],marker_color=colors[13],
                     legendgroup='1',
                     name='Arthropoda'),row=3,col=1)


color_index = [2,4,6,8,10,12,13]
for i,animal in enumerate(['Echinodermata', 'Bryozoa','Brachiopoda','Mollusca', 'Porifera',  'Cnidaria','Arthropoda']):

    color = colors[color_index[i]] if color_index[i]!= 8 else '#065535'

    fig.add_trace(go.Bar(x=animal_relative_occurrence_phylum['index'],
                         y=animal_relative_occurrence_phylum[animal],marker_color=color,
                         name=f'{animal}'),row=4,col=1)


fig.update_yaxes(title = 'Relative skeletal<br>biomass %',ticks = 'outside',showline = True, 
                         mirror = True,linecolor = 'black',row=1,col=1)
fig.update_yaxes(title = 'Relative skeletal<br>occurrence %',ticks = 'outside',showline = True, 
                         mirror = True,linecolor = 'black',row=2,col=1)
fig.update_yaxes(title = 'Relative animal<br>biomass %',ticks = 'outside',showline = True, 
                         mirror = True,linecolor = 'black',row=3,col=1)
fig.update_yaxes(title = 'Relative animal<br>occurrence %',ticks = 'outside',showline = True, 
                         mirror = True,linecolor = 'black',row=4,col=1)


fig.update_xaxes(ticks = 'outside',showticklabels = False,showline = True, 
                     mirror = True,linecolor = 'black',row=1,col=1)
fig.update_xaxes(ticks = 'outside',showticklabels = False,showline = True, 
                     mirror = True,linecolor = 'black',row=2,col=1)
fig.update_xaxes(ticks = 'outside',showticklabels = False,showline = True, 
                     mirror = True,linecolor = 'black',row=3,col=1)
fig.update_xaxes(ticks = 'outside',showticklabels = True,showline = True, tickangle=-60,
                     mirror = True,linecolor = 'black',row=4,col=1)



fig.update_layout(height = 1450, width = 950,font_color = 'black', font_family='Helvetica',
                  font_size=14,
                  barmode='stack',
                  legend=dict(orientation="v",
                                yanchor="top",
                                y=0.99,
                                xanchor="left",x=1.02, font_size=14
                                ),
                  legend_tracegroupgap = 550,
                 paper_bgcolor = 'white',
                 plot_bgcolor = 'white',)
fig.show()

# fig.write_image('./figures/latest/Figure_3.jpeg',scale=6)